In [131]:
import pandas as pd
import numpy as np

## For plotting
import matplotlib.pyplot as plt
import seaborn as sns

## This sets the plot style
## to have a grid on a dark background
sns.set_style("whitegrid")

In [132]:

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score, confusion_matrix




In [133]:
# Read in your csv file that has the mid features. 

data = pd.read_csv('../Feature Extraction/midFeaturesTrainSet.csv')
data.head()

print(data.shape)

(4877, 140)


In [134]:
# Split the data into train and test set, stratified by Emotion.


data_train, data_test = train_test_split(data.copy(),
                                   shuffle=True,
                                   random_state=608,
                                   stratify=data.Emotion,
                                   test_size=0.2
                                   )

In [135]:
data.sample(5)

,FileID,ActorID,Emotion,SentenceID,zcr_mean,energy_mean,energy_entropy_mean,spectral_centroid_mean,spectral_spread_mean,spectral_entropy_mean,...,delta chroma_4_std,delta chroma_5_std,delta chroma_6_std,delta chroma_7_std,delta chroma_8_std,delta chroma_9_std,delta chroma_10_std,delta chroma_11_std,delta chroma_12_std,delta chroma_std_std
1694,1057_DFA_SAD_XX,1057,SAD,DFA,0.061546,0.019819,3.085951,0.163357,0.232544,0.224647,...,0.025297,0.013702,0.013433,0.039907,0.002901,0.009223,0.012299,0.026799,0.005323,0.009926
1740,1017_WSI_DIS_XX,1017,DIS,WSI,0.054826,0.012612,3.053301,0.157795,0.215537,0.323212,...,0.022220,0.013250,0.010048,0.023185,0.006262,0.011540,0.016258,0.021833,0.004337,0.008145
2618,1059_ITS_NEU_XX,1059,NEU,ITS,0.061341,0.014192,3.053282,0.163517,0.228778,0.301374,...,0.025445,0.018096,0.012015,0.039817,0.003159,0.007975,0.015709,0.028962,0.003413,0.009880
3008,1024_ITS_SAD_XX,1024,SAD,ITS,0.069962,0.013972,3.065955,0.163294,0.218506,0.454811,...,0.016825,0.013332,0.007600,0.041061,0.007034,0.004815,0.012639,0.022336,0.011901,0.009753
4113,1010_TSI_HAP_XX,1010,HAP,TSI,0.065695,0.016530,3.037603,0.161804,0.212808,0.356078,...,0.020422,0.022628,0.015900,0.025028,0.007062,0.007379,0.009488,0.027628,0.022782,0.010083


In [136]:
# Check the percentages of the different emotion categories in the training set



data_train.Emotion.value_counts(normalize=True)

NEU    0.178672
SAD    0.164317
HAP    0.164317
DIS    0.164317
FEA    0.164317
ANG    0.164060
Name: Emotion, dtype: float64

In [137]:
# Check the percentages of the different emotion categories in the test set

data_test.Emotion.value_counts(normalize=True)

NEU    0.179303
ANG    0.164959
DIS    0.163934
SAD    0.163934
HAP    0.163934
FEA    0.163934
Name: Emotion, dtype: float64

In [138]:
# Separate the train and test(validataion) set based on the sentences 

# IEO - It's 11 o'clock.
# TIE - That is exactly what happend.
# IOM - I'm on my way to the meeting.
# IWW - I wonder what this is about.
# TAI - The airplane is almost full.
# MTI - Maybe tomorrow it will be cold.
# IWL - I would like a new alarm clock.
# ITH - I think I have a doctor's appointment.
# DFA - Dont forget a jacket.
# ITS - I think I've seen this before.
# TSI - The surface is slick.
# WSI - We'll stop in a couple of minutes.

# Get the rows of X_train, X_test corresponding to specific sentences

# Split the train and test set into labels (y) and features (X)

y_train = data_train[['Emotion']]
y_test  = data_test[['Emotion']]

#X_train = data_train.drop(columns  = ['ActorID', 'Emotion', 'SentenceID'])
#X_test  = data_test.drop(columns   = ['ActorID', 'Emotion', 'SentenceID'])



In [139]:
X_train = data_train[['FileID','energy_mean']]
X_test  = data_test[['FileID','energy_mean']]

In [140]:
# Separate the train and test(validataion) set based on the sentences 

# IEO - It's 11 o'clock.
# TIE - That is exactly what happend.
# IOM - I'm on my way to the meeting.
# IWW - I wonder what this is about.
# TAI - The airplane is almost full.
# MTI - Maybe tomorrow it will be cold.
# IWL - I would like a new alarm clock.
# ITH - I think I have a doctor's appointment.
# DFA - Dont forget a jacket.
# ITS - I think I've seen this before.
# TSI - The surface is slick.
# WSI - We'll stop in a couple of minutes.


X_train.FileID = X_train["FileID"].str[5:8]
X_test.FileID  = X_test["FileID"].str[5:8]

#X_train_sen = X_train.loc[(X_train.FileID == 'IWW')]
#X_test_sen  = X_test.loc[(X_test.FileID == 'IWW')]

#y_train_sen = y_train.loc[(X_train.FileID == 'IWW')]
#y_test_sen  = y_test.loc[(X_test.FileID == 'IWW')]

X_train_sen = X_train
X_test_sen  = X_test

y_train_sen = y_train
y_test_sen = y_test

/home/mnoorani/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [141]:
X_train_sen

,FileID,energy_mean
3831,MTI,0.019216
3591,TSI,0.013772
407,TSI,0.024157
4372,WSI,0.005302
2977,DFA,0.015481
...,...,...
3493,ITH,0.019385
3584,ITS,0.022356
2414,DFA,0.020709
3855,TAI,0.016790


In [142]:
X_train_sen = X_train_sen.drop(columns = ['FileID'])
X_test_sen  = X_test_sen.drop(columns = ['FileID'])


In [143]:
# Add a column to the y vectors encoding each of the emotions.

y_train_sen_dummies = pd.get_dummies(y_train_sen)
y_train_sen         = pd.concat([y_train_sen, y_train_sen_dummies], axis=1)

y_test_sen_dummies  = pd.get_dummies(y_test)
y_test_sen          = pd.concat([y_test_sen, y_test_sen_dummies], axis=1)


In [144]:
# Check that the encoding looks right

y_train_sen

,Emotion,Emotion_ANG,Emotion_DIS,Emotion_FEA,Emotion_HAP,Emotion_NEU,Emotion_SAD
3831,SAD,0,0,0,0,0,1
3591,ANG,1,0,0,0,0,0
407,HAP,0,0,0,1,0,0
4372,SAD,0,0,0,0,0,1
2977,DIS,0,1,0,0,0,0
...,...,...,...,...,...,...,...
3493,ANG,1,0,0,0,0,0
3584,ANG,1,0,0,0,0,0
2414,FEA,0,0,1,0,0,0
3855,FEA,0,0,1,0,0,0


In [145]:
y_train_sen.Emotion[y_train_sen.Emotion == 'NEU']

3674    NEU
4594    NEU
3292    NEU
2040    NEU
4808    NEU
       ... 
170     NEU
2812    NEU
3816    NEU
3565    NEU
3857    NEU
Name: Emotion, Length: 697, dtype: object

In [176]:
# Example: try to train a support vector machine with Gaussian radial kernel to distinguish between instances
# where emotion is 'ANG' and instances where emotion is 'HAP'. 

# Get the rows of X_train, X_test corresponding to just the emotions 'ANG' and 'HAP'

X_train_sub = X_train_sen.loc[(y_train_sen.Emotion == 'FEA') | (y_train_sen.Emotion == 'NEU')]
X_test_sub  = X_test_sen.loc[(y_test_sen.Emotion == 'FEA') | (y_test_sen.Emotion == 'NEU')]


# Get the Emotion_ANG column of the ys, with only the rows corresponding to 'ANG' and 'HAP'

y_train_sub = y_train_sen.loc[(y_train_sen.Emotion == 'FEA') | (y_train_sen.Emotion == 'NEU')].Emotion_FEA
y_test_sub = y_test_sen.loc[(y_test_sen.Emotion == 'FEA') | (y_test_sen.Emotion == 'NEU')].Emotion_FEA



In [177]:
#print("X_train:", X_train_sub)
print("y_train",  y_train_sub)

y_train 1528    1
2254    1
2332    1
3674    0
2641    1
       ..
3565    0
333     1
3857    0
2414    1
3855    1
Name: Emotion_FEA, Length: 1338, dtype: uint8


In [178]:
np.unique(y_test_sub)

array([0, 1], dtype=uint8)

In [179]:
# Build pipeline to first scale the mid feature data, then apply the SVC

pipe = Pipeline([('scale', StandardScaler()),
                 ('svc', SVC(kernel='rbf'))])


# Fit the model to the training data

pipe.fit(X_train_sub, y_train_sub)

    
# Get the model's prediction on the test data

pred = pipe.predict(X_test_sub)



In [180]:
# Build pipeline to first scale the mid feature data, then apply the SVC

pipe = Pipeline([('scale', StandardScaler()),
                 ('svc', SVC(kernel='rbf'))])


# Fit the model t#o the training data

pipe.fit(X_train_sub, y_train_sub)

    
# Get the model's prediction on the test data

pred = pipe.predict(X_test_sub)


In [181]:
# Look at the confusion matrix for the test data :

print("confusion matrix for the test set is:")
print(confusion_matrix(y_test_sub, pred))
print()

# Look at the confusion matrix for the training data:
pred_train = pipe.predict(X_train_sub)
print("confusion matrix for the train set is:")
print(confusion_matrix(y_train_sub, pred_train))


confusion matrix for the test set is:
[[148  27]
 [122  38]]

confusion matrix for the train set is:
[[572 125]
 [505 136]]
